In [ ]:
import time, psutil
uptime = time.time() - psutil.boot_time()
print('How much I used?\n {} hours, and {:.2f} minutes '.format(uptime//3600, uptime%60))
remain = 12*60*60 - uptime
print('How much is remaining?\n {} hours, and {:.2f} minutes '.format(remain//3600, remain%60))


How much I used?
 1.0 hours, and 12.07 minutes 
How much is remaining?
 10.0 hours, and 47.93 minutes 


# Evaluator Packages

In [ ]:
# Install required packages

!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

     |████████████████████████████████| 1.8MB 6.8MB/s 
     |████████████████████████████████| 3.2MB 41.3MB/s 
     |████████████████████████████████| 890kB 48.7MB/s 
     |████████████████████████████████| 317kB 5.0MB/s 
     |████████████████████████████████| 235kB 9.4MB/s 
     |████████████████████████████████| 1.4MB 10.5MB/s 
     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 133kB 10.2MB/s 
     |████████████████████████████████| 245kB 40.7MB/s 


In [ ]:
# Import required packages

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm
from itertools import chain

import os
import re
import json
import copy
import collections

from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features
import math 
import tensorflow as tf
from termcolor import colored

from sklearn.model_selection import StratifiedKFold
from statistics import mean
import glob
import os
import glob
import string

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH)

# Getting the annotations data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

who_am_I = 'mitra' # mitra or reza
if who_am_I == 'mitra':
    data_address = '/content/drive/MyDrive/COVID-PSS.xls'
    keys_address = '/content/drive/MyDrive/keywords_final_distilled_NE (1).pickle'
    cleaned_titles_address = '/content/drive/MyDrive/title_cleaned_without_corona_2.pkl'
else:
    data_address = None
    keys_address = None
    cleaned_titles_address = None
df = pd.read_csv(data_address)
list_t = pd.read_pickle(cleaned_titles_address)

keywords = pd.read_pickle(keys_address)
keywords = [v for k,v in keywords.items()]



assert len(keywords) == len(df)
df['keywords'] = keywords
df.drop(columns=['img', 'link'], inplace=True)


# annotations
df_annotations = pd.read_csv('/content/drive/MyDrive/all_annotations.csv')
df_annotations.head()
train_test_split = 2500
first_sentences = df_annotations.sentence1.values.tolist()[:train_test_split]
second_sentences = df_annotations.sentence2.values.tolist()[:train_test_split]
# df_annotations.loc[:, 'score'] = df_annotations.loc[:, 'score'].apply(lambda x: np.round(x-1))

# Helper

In [ ]:
things_to_notice = '،«»”“؛؟'
punct = string.punctuation + things_to_notice


normalizer = hazm.Normalizer()

def clean(text):

    text = re.sub('کووید-19 ', 'کرونا', text)
    text = text.translate(str.maketrans('', '', punct))
    text = re.sub(' +[\w] +', " ", text)
    
    text = normalizer.normalize(text)

    return text

cleaned_titles = df.title.apply(lambda x : clean(x))


# ----------------- precision--------------------------------

def cal_precision(inp):
  return sum(inp) / len(inp)

def cal_AP(inp):
  Rs = [i for i, each in enumerate(inp) if each == 1]
  P_Rs = []
  for r in Rs:
    P_Rs.append(cal_precision(inp[:r + 1]))
  if sum([i==0 for i in P_Rs])==len(P_Rs):
      return(0)
  return mean(P_Rs)

# ----------------- dcj --------------------------------#


def cal_idcg(inp):
  tmp = inp[0]
  for i, each in enumerate(inp[1:]):
    idx = i + 2
    tmp += each / math.log2(idx)
  return tmp


def cal_ndcg(inp):
  dcg, idcg = cal_idcg(inp), cal_idcg(sorted(inp, reverse=True))
  return dcg / idcg


# -------------------function that calculates the results --------------------------# 

results_path = r'/content/drive/MyDrive/CoPer paper-Models/Results/'

def show_results_with_records(model_name, top_n=10, results_path=results_path, minimum_score_for_precision=0.5, print_output=False):
    """ gets a model name and calculates the results """

    print(colored('Model: ', 'blue'), model_name)

    results = pd.read_pickle(results_path + model_name + '.pkl')
    # average of all 
    averages_0_1 = []
    averages_0_4 = []
    predictions_0_4 = []
    predictions_0_1 = []
    more_than_mins = []
    precisions = []
    aps = []
    ndcjs = []


    i = 0
    for record in results:
        
        # getting the question and the index of related records
        question = record['question']
        indices = record['index'][:top_n]

        
        test_inputs = bert_encode_customized([question]*len(indices),
                                        [list_t[int(i)] for i in indices],
                                        tokenizer)

        # appending the predictions to a list
        # predictions
        prediction = [max(i[0], 0) for i in model.predict(test_inputs)]
        prediction_s = [np.ceil(j*4) for j in prediction]


        predictions_0_1.append(prediction)
        predictions_0_4.append(prediction_s)

        # calculate ndcj with the scaled score
        ndcj = cal_ndcg(prediction_s, )
        ndcjs.append(ndcj)

        # add the average of predictions for each query
        averages_0_1.append(np.average(predictions_0_1))
        averages_0_4.append(np.average(predictions_0_4))


        # calculating records that are correctly identified as being similar
        # ones that have scores more than minimum are relevant and ones with lower 
        # scores are 0

        more_than_min = [1 if m>=minimum_score_for_precision else 0 for m in prediction]
        len_more_than_min = sum(more_than_min)
        

        # length of the list of related documents
        more_than_mins.append(len_more_than_min)


        # one precision for this query is calculated
        precision_query_i = len_more_than_min / int(len(indices))

        # adding the precision for this question to a list of all precisions
        precisions.append(precision_query_i)


        
        # calculate the ap for each query
        ap_i = cal_AP(more_than_min)

        # add this to the aps list for getting the average
        aps.append(ap_i)






        if print_output == True:   
            print(more_than_min)
            print(colored(f'question {i}: {question}', 'green'))

            for j in range(len(indices)):
                print(colored(f'{j} :', 'red'), cleaned_titles[int(indices[j])], colored('\nscore:', 'red'), prediction[j], '\n')
            
        
        i+=1


    avg_of_all_avgs_0_1 = np.average(averages_0_1)
    avg_of_all_avgs_0_4 = np.average(averages_0_4)
    avg_of_all_prec = np.average(precisions)
    avg_of_all_mtm = np.average(more_than_mins)
    avg_of_aps = np.average(aps)
    avg_ndcj = np.average(ndcjs)

    return {
        # predictions in the scale of 0 to 1 and 0 to 4
        #'predictions scale 1': predictions_0_1,
        #'predictions scale 4': predictions_0_4,

        # mean of averages for all questions that will be one number
        'mean of the mean score of all questions scale 0-1': avg_of_all_avgs_0_1,
        'mean of the mean score of all questions scale 0-4': avg_of_all_avgs_0_4,
        
        # f'average number of records with score more than {minimum_score_for_precision}' : avg_of_all_mtm,
        f'MP@{top_n}' : avg_of_all_prec,

        'Mean NDCJ':avg_ndcj,
        f'MAP@{top_n}': avg_of_aps
        }

NameError: ignored

In [ ]:
cleaned_titles

0               وضعیت کرونایی هفت شهرستان مازندران زرد شد
1       خطر انتقال کرونا در عابربانک‌ها توصیه‌های وزار...
2                                       مگر کرونا تمام شد
3       ورود بیش از ۴۰ هزار نفر به هرمزگان احتمال وقوع...
4                                       گردش دوباره کرونا
                              ...                        
3531      تاثیر مخرب خبرهای منفی کرونا بر روح روان کودکان
3532    ۱۲ راهکار ساده برای مقابله با احساسات منفی ناش...
3533      استرس کرونا خیلی بیشتر از واقعیت‌های بیماری است
3534                    پرهیز از وسواس در مواجهه با کرونا
3535    آمار جان‌باختگان کرونا جزی‌تر اندک‌تر از روان ...
Name: title, Length: 3536, dtype: object

# Creating the inputs

In [ ]:
max_seq_length=128 # the longer the training len the less accurate the resutls

In [ ]:
def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
def bert_encode_customized(first_sentences, second_sentences, tokenizer, max_len=max_seq_length):

    input_type_ids = []
    input_mask = []
    input_word_ids = []

    sentence1 = [encode_sentence(s) for s in np.array(first_sentences)]
    sentence2 = [encode_sentence(s) for s in np.array(second_sentences)]

    for i in range(len(first_sentences)):

        cls = [tokenizer.convert_tokens_to_ids('[CLS]')]
        input_id_i = cls + sentence1[i] + sentence2[i]
        length_i = len(input_id_i)
        input_id_i =  input_id_i + (max_len - length_i) * [0]

        input_masks_i = length_i * [1] + (max_len - length_i) * [0]
        input_type_id_i = [0] + [0] * len(sentence1[i]) + [1] * len(sentence2[i]) + (max_len - length_i) * [0]
        
        input_word_ids.append(input_id_i[:max_seq_length])
        input_type_ids.append(input_type_id_i[:max_seq_length])
        input_mask.append(input_masks_i[:max_seq_length])

    inputs = {
      'input_word_ids': tf.ragged.constant(input_word_ids).to_tensor(),
      'input_mask': tf.ragged.constant(input_mask).to_tensor(),
      'input_type_ids': tf.ragged.constant(input_type_ids).to_tensor()}
    return inputs

In [ ]:
train_input_customized = bert_encode_customized(first_sentences, second_sentences, tokenizer, max_len=max_seq_length)

# Building the Evaluator Model

In [ ]:
max_len = max_seq_length

def build_custome_model():
    bert_encoder = TFBertModel.from_pretrained(MODEL_NAME_OR_PATH)
    
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")
    
    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]

    clf_output = embedding[:,0,:]

    #net = tf.keras.layers.Dense(32, activation='tanh')(clf_output)
    net = tf.keras.layers.Dropout(0.5)(clf_output)

    # output = tf.keras.layers.Dense(5, activation='softmax')(net)

    output = tf.keras.layers.Dense(1)(net)

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)

    
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss= tf.keras.losses.MeanSquaredError(), metrics = ['mse'])
    
    return model

In [ ]:
Tpu = 'on'
if Tpu == 'on':
        # a few lines of code to get our tpu started and our data distributed on it
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    # print("All devices: ", tf.config.list_logical_devices('TPU'))

    strategy = tf.distribute.experimental.TPUStrategy(resolver)
    

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.123.247.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.123.247.26:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
with strategy.scope():
        model = build_custome_model()
        model.summary()

Some layers from the model checkpoint at HooshvareLab/bert-fa-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f97592c02a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f97592c02a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f97592c02a0> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert

Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 162841344   input_word_ids[0][0]             
                                                                 input_mask[0][0]             

Current Method (Regression)

In [ ]:
k = 5
kfold = StratifiedKFold(n_splits = k, shuffle = True)

df_train = df_annotations.copy()

labels = df_train.loc[:train_test_split-1, 'score'].apply(lambda x: (x-1)/4)

round_labels = df_train.loc[:train_test_split-1, 'score'].apply(lambda x: np.round(x))


In [ ]:

for i, (train_idx, val_idx) in enumerate(kfold.split(second_sentences, round_labels), 1):
    print(colored(f'fold {i}', 'blue'))
    epoch_evaluation = {}
    print(train_idx)
    train_input_customized = bert_encode_customized(np.array(first_sentences)[train_idx], np.array(second_sentences)[train_idx], tokenizer, max_len=max_seq_length)
    validation_input_customized = bert_encode_customized(np.array(first_sentences)[val_idx], np.array(second_sentences)[val_idx], tokenizer, max_len=max_seq_length)

    history = model.fit(x = train_input_customized, y = labels.loc[train_idx],
                        validation_data= (validation_input_customized, labels.loc[val_idx]),
                        epochs = 5,
                        verbose = 1,
                        batch_size = 8)

    epoch_evaluation['Fold '+ str(i)] = history.history

fold 1
[   1    2    3 ... 2496 2497 2499]
Epoch 1/5


250/250 [==============================] - ETA: 0s - loss: 1.6313 - mse: 1.6313WARNING:tensorflow:The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


250/250 [==============================] - 78s 99ms/step - loss: 1.6296 - mse: 1.6296 - val_loss: 0.0911 - val_mse: 0.0911
Epoch 2/5
250/250 [==============================] - 19s 77ms/step - loss: 0.6898 - mse: 0.6898 - val_loss: 0.0889 - val_mse: 0.0889
Epoch 3/5
250/250 [==============================] - 19s 77ms/step - loss: 0.4621 - mse: 0.4621 - val_loss: 0.0783 - val_mse: 0.0783
Epoch 4/5
250/250 [==============================] - 19s 76ms/step - loss: 0.3806 - mse: 0.3806 - val_loss: 0.0465 - val_mse: 0.0465
Epoch 5/5
250/250 [==============================] - 19s 76ms/step - loss: 0.2489 - mse: 0.2489 - val_loss: 0.0328 - val_mse: 0.0328
fold 2
[   0    1    2 ... 2494 2498 2499]
Epoch 1/5
250/250 [==============================] - 19s 77ms/step - loss: 0.2052 - mse: 0.2052 - val_loss: 0.0296 - val_mse: 0.0296
Epoch 2/5
250/250 [==============================] - 19s 77ms/step - loss: 0.1636 - mse: 0.1636 - val_loss: 0.0417 - val_mse: 0.0417
Epoch 3/5
250/250 [=================

# Getting the Results

In [ ]:

basePath = r"/content/drive/MyDrive/CoPer paper-Models/Results"

filePaths = glob.glob(os.path.join(basePath,'*{0}*.pkl'))

# Just open first ocurrence, if any
if filePaths:
    print ("Found: ", filePaths[0])
    doc = open(filePaths[0], 'r')

In [ ]:
results_path = r'/content/drive/MyDrive/CoPer paper-Models/Results/'

basePath = results_path + '*'

print(colored('The indices are ready for:', 'blue'))
for fpath in glob.glob(basePath):
    length_ = len(results_path)
    print (fpath[length_:][:-4])

The indices are ready for:
sbert-WikiNli-DifferentWeights_FineTuned_f300-Ranked_2
Ranked_BM25_TFIDF_sbert-WikiNli-DifferentWeights_FineTuned_f500_Ranked
AlBert
TFIDF
sbert-WikiNli
BM25
BM25_PrimaryRanking
sbert-WikiNli-DifferentWeights-Ranked
TFIDF_Ranked
Ranked_BM25_TFIDF_SBERT_Wikinli_Fine-tuned_Different-Weights
sbert-WikiNli-FineTuned
sbert-WikiNli-FineTuned_100
sbert-WikiNli-FineTuned_200
sbert-WikiNli-FineTuned_300
sbert-WikiNli-FineTuned_500
sbert-WikiNli-DifferentWeights_FineTuned_f100-Ranked
Ranked_BM25_TFIDF_sbert-WikiNli-DifferentWeights_FineTuned_f100-Ranked
Ranked_BM25_TFIDF_sbert-WikiNli-DifferentWeights_FineTuned_f100_0.5toTFIDF-Ranked
Ranked_BM25_TFIDF_sbert-WikiNli-DifferentWeights_FineTuned_f100_0.25toTFIDF-Ranked
BERT
sbert-WikiTriplet
sbert-farsTail
Ranked_BM25_TFIDF_sbert-WikiNli-DifferentWeights_FineTuned_f500_Ranked_2
Ranked_BM25_TFIDF_sbert-WikiNli-DifferentWeights_FineTuned_f300_Ranked_2
Ranked_BM25_TFIDF_sbert-WikiNli-DifferentWeights_FineTuned_f300_Ranked_2-0

# Exp with the Results

In [ ]:
#-----------first

sbert_WikiNli_results = show_results_with_records('sbert-WikiNli', top_n=10, results_path=results_path, minimum_score_for_precision=0.4) # new questions
sbert_WikiNli_results

Model:  sbert-WikiNli


{'MAP@10': 0.6776703357268833,
 'MP@10': 0.494,
 'Mean NDCJ': 0.9194348154076143,
 'mean of the mean score of all questions scale 0-1': 0.4113818288054191,
 'mean of the mean score of all questions scale 0-4': 2.1560595691859303}

In [ ]:
#----------second

sbert_WikiNli_results = show_results_with_records('sbert-WikiNli', top_n=20, results_path=results_path, minimum_score_for_precision=0.4) # new questions
sbert_WikiNli_results

Model:  sbert-WikiNli


{'MAP@20': 0.5726935601550683,
 'MP@20': 0.40199999999999997,
 'Mean NDCJ': 0.894720851190815,
 'mean of the mean score of all questions scale 0-1': 0.3693602134815125,
 'mean of the mean score of all questions scale 0-4': 1.9923014541557655}

In [ ]:
#-----------first

sbert_wikinli_ft_100 = show_results_with_records('sbert-WikiNli-FineTuned_100', top_n=10, results_path=results_path, minimum_score_for_precision=0.4) # new questions
sbert_wikinli_ft_100

Model:  sbert-WikiNli-FineTuned_100


{'MAP@10': 0.6732532375913328,
 'MP@10': 0.502,
 'Mean NDCJ': 0.9175981303851016,
 'mean of the mean score of all questions scale 0-1': 0.41674573051224356,
 'mean of the mean score of all questions scale 0-4': 2.171487607845909}

### different fine tuneds

In [ ]:
sbert_wikinli_ft_200 = show_results_with_records('sbert-WikiNli-FineTuned_200', top_n=10, results_path=results_path, minimum_score_for_precision=0.4) # new questions
sbert_wikinli_ft_200

Model:  sbert-WikiNli-FineTuned_200


{'MAP@10': 0.6732532375913328,
 'MP@10': 0.502,
 'Mean NDCJ': 0.9175981303851016,
 'mean of the mean score of all questions scale 0-1': 0.41674573051224356,
 'mean of the mean score of all questions scale 0-4': 2.171487607845909}

In [ ]:
sbert_wikinli_ft_300 =  show_results_with_records('sbert-WikiNli-FineTuned_300', top_n=10, results_path=results_path, minimum_score_for_precision=0.4) # new questions
sbert_wikinli_ft_300

Model:  sbert-WikiNli-FineTuned_300


{'MAP@10': 0.6732532375913328,
 'MP@10': 0.502,
 'Mean NDCJ': 0.9175981303851016,
 'mean of the mean score of all questions scale 0-1': 0.41674573051224356,
 'mean of the mean score of all questions scale 0-4': 2.171487607845909}

In [ ]:
sbert_wikinli_ft_500 =  show_results_with_records('sbert-WikiNli-FineTuned_500', top_n=10, results_path=results_path, minimum_score_for_precision=0.4) # new questions
sbert_wikinli_ft_500

Model:  sbert-WikiNli-FineTuned_500


{'MAP@10': 0.6730310153691106,
 'MP@10': 0.502,
 'Mean NDCJ': 0.9175786417703057,
 'mean of the mean score of all questions scale 0-1': 0.41674573051224356,
 'mean of the mean score of all questions scale 0-4': 2.171487607845909}

# others

In [ ]:
TFIDF = show_results_with_records('TFIDF', top_n=10, results_path=results_path, minimum_score_for_precision=0.4) # new questions
TFIDF

Model:  TFIDF
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
question 0: آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0 : تزریق واکسن کرونا در بارداری بی خطر است 
score: 0.5976458 

1 : خطرات تجویز داروهای کرونا برای زنان باردار 
score: 0.6448008 

2 : زنان باردار درمقابل کرونا چه کارهایی باید انجام دهند 
score: 0.64570296 

3 : کروناویروس جدید از مادر به جنین منتقل نمی‌شود 
score: 0.577409 

4 : مطالعات جدید اثر کرونا بر زنان باردار به چه صورت است 
score: 0.6614979 

5 : توصیه‌های وزارت بهداشت به زنان باردار در روزهای کرونایی 
score: 0.6478202 

6 : توصیه هایی برای پیشگیری از کرونا در زنان باردار 
score: 0.68010855 

7 : کرونا لزوم مراقبت‌های بهداشتی در دوران بارداری 
score: 0.639253 

8 : آیا بهبودیافتگان کرونا هم باید واکسن بزنند 
score: 0.3987338 

9 : آیا کرونا از مادر به جنین منتقل می‌شود 
score: 0.74008846 

[0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
question 1: من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
0 : ۷ باور غلط که در روز‌های کرونایی سلامتی‌تان را تهدید می

{'MAP@10': 0.802930612244898,
 'MP@10': 0.6559999999999999,
 'Mean NDCJ': 0.9330909685228939,
 'mean of the mean score of all questions scale 0-1': 0.50953007,
 'mean of the mean score of all questions scale 0-4': 2.5132419876446845,
 'predictions scale 1': [[0.5976458,
   0.6448008,
   0.64570296,
   0.577409,
   0.6614979,
   0.6478202,
   0.68010855,
   0.639253,
   0.3987338,
   0.74008846],
  [0.3630014,
   0.39404577,
   0.17151093,
   0.29312953,
   0.1541132,
   0.4569338,
   0.6942357,
   0.37980217,
   0.064991534,
   0.40963146],
  [0.7025567,
   0.6355395,
   0.48373127,
   0.6994442,
   0.53471434,
   0.6619868,
   0.57089186,
   0.7121575,
   0.64739555,
   0.60653985],
  [0.827912,
   0.6118312,
   0.7390993,
   0.6733267,
   0.67491794,
   0.6704146,
   0.6910368,
   0.66480803,
   0.21248499,
   0.8320115],
  [0.5967498,
   0.8737753,
   0.6834991,
   0.28718385,
   0.3049146,
   0.7646746,
   0.8384964,
   0.6644592,
   0.7688628,
   0.8121196],
  [0.75769186,
   0.69

In [ ]:
sbert_WikiNli_results = show_results_with_records('sbert-WikiNli', top_n=10, results_path=results_path, minimum_score_for_precision=0.4) # new questions
sbert_WikiNli_results

Model:  sbert-WikiNli
[1, 0, 1, 1, 0, 0, 0, 0, 1, 0]
question 0: آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0 : سیگاری‌ها بیشتر در خطر ابتلا به کرونا هستند 
score: 0.43276945 

1 : مصرف‌کنندگان موادمخدر بیشتر در معرض کرونا هستند 
score: 0.37814552 

2 : احتمال ابتلا به کرونا در افراد طاس بیشتر است 
score: 0.4229703 

3 : کدام گروه سنی بیشتر در معرض خطر کروناست 
score: 0.58781886 

4 : احتمال ابتلای‌ قدبلندها به کرونا بیشتر است 
score: 0.27956587 

5 : خطر ابتلا به کرونا در افراد کم خواب 
score: 0.3181726 

6 : کودکان کرونا نمی‌گیرند این باور غلط آنها را به خطر می‌اندازد 
score: 0.26913902 

7 : ۴۲ درصد مبتلایان کرونا جوان هستند 
score: 0.26441994 

8 : امکان ابتلای شدید به کرونا برای کسانی که قبلا مبتلا شده‌اند 
score: 0.4400134 

9 : وجود بیماری‌های زمینه‌ای در ۷۵ درصد از جوانان فوت شده بر اثر کرونا 
score: 0.118169315 

[0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
question 1: من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
0 : طلوع زندگی پس از کرونا 
score:

{'MAP@10': 0.6776703357268833,
 'MP@10': 0.494,
 'Mean NDCJ': 0.9194348154076143,
 'mean of the mean score of all questions scale 0-1': 0.4113818288054191,
 'mean of the mean score of all questions scale 0-4': 2.1560595691859303}

In [ ]:
sbert_WikiNli_Finetuned_results = show_results_with_records('sbert-WikiNli-FineTuned', top_n=10, results_path=results_path, minimum_score_for_precision=0.4) # new questions
sbert_WikiNli_Finetuned_results

Model:  sbert-WikiNli-FineTuned
[1, 1, 1, 1, 0, 0, 1, 0, 1, 1]
question 0: آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0 : سیگاری‌ها بیشتر در خطر ابتلا به کرونا هستند 
score: 0.50650954 

1 : مصرف‌کنندگان موادمخدر بیشتر در معرض کرونا هستند 
score: 0.51240385 

2 : احتمال ابتلای‌ قدبلندها به کرونا بیشتر است 
score: 0.4719077 

3 : احتمال ابتلا به کرونا در افراد طاس بیشتر است 
score: 0.52330977 

4 : آیا سیگاری‌ها معتادان به کرونا مبتلا نمی‌شوند 
score: 0.38531637 

5 : مرگ میر بر اثر کرونا در کدام بیماران بیشتر است 
score: 0.30853167 

6 : ۴۲ درصد مبتلایان کرونا جوان هستند 
score: 0.40496105 

7 : وجود بیماری‌های زمینه‌ای در ۷۵ درصد از جوانان فوت شده بر اثر کرونا 
score: 0.23662229 

8 : کدام گروه سنی بیشتر در معرض خطر کروناست 
score: 0.66398525 

9 : آیا اختلالات تیروئید ریسک ابتلا به کروناویروس را افزایش می‌دهد 
score: 0.47757724 

[0, 1, 0, 1, 1, 1, 1, 0, 0, 1]
question 1: من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
0 : طلوع زندگی پس از کرو

{'MAP@10': 0.7568966301335348,
 'MP@10': 0.68,
 'Mean NDCJ': 0.9341133942195795,
 'mean of the mean score of all questions scale 0-1': 0.49463657,
 'mean of the mean score of all questions scale 0-4': 2.5071005895573775,
 'predictions scale 1': [[0.50650954,
   0.51240385,
   0.4719077,
   0.52330977,
   0.38531637,
   0.30853167,
   0.40496105,
   0.23662229,
   0.66398525,
   0.47757724],
  [0.34750092,
   0.5828557,
   0.28390542,
   0.42512608,
   0.49080524,
   0.5609355,
   0.51428175,
   0.18912816,
   0.25854045,
   0.53925484],
  [0.44440496,
   0.6994442,
   0.60653985,
   0.6427395,
   0.5523268,
   0.3450975,
   0.4089861,
   0.50051177,
   0.21443713,
   0.274422],
  [0.7127556,
   0.6783359,
   0.827912,
   0.45573378,
   0.586103,
   0.66480803,
   0.8320115,
   0.453519,
   0.5316352,
   0.6910368],
  [0.7688628,
   0.37471625,
   0.26482502,
   0.3381585,
   0.7981358,
   0.46681684,
   0.3403681,
   0.76257306,
   0.49111,
   0.5253413],
  [0.76480377,
   0.7135135,
 

In [ ]:
sbert_WikiNli_Finetuned_results = show_results_with_records('Ranked_BM25_TFIDF_SBERT_Wikinli_Fine-tuned_Different-Weights', top_n=10, results_path=results_path, minimum_score_for_precision=0.4) # new questions
sbert_WikiNli_Finetuned_results

Model:  Ranked_BM25_TFIDF_SBERT_Wikinli_Fine-tuned_Different-Weights
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
question 0: آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0 : سیگاری‌ها بیشتر در خطر ابتلا به کرونا هستند 
score: 0.50650954 

1 : خطر مرگ ابتلا به کرونا در کودکان چاق بیشتر است 
score: 0.51525116 

2 : آیا کودکان چاق مستعد ابتلا به کرونا هستند 
score: 0.5784531 

3 : خطر ابتلا به سکته مغزی در افراد سیگاری مبتلا به کرونا بیشتر است 
score: 0.47669148 

4 : احتمال ابتلا به کرونا در افراد طاس بیشتر است 
score: 0.52330977 

5 : معرفی پرریسک‌ترین افراد در معرض خطر ابتلا به کرونا 
score: 0.4945562 

6 : کدام گروه سنی بیشتر در معرض خطر کروناست 
score: 0.66398525 

7 : احتمال ابتلای‌ قدبلندها به کرونا بیشتر است 
score: 0.4719077 

8 : مصرف‌کنندگان موادمخدر بیشتر در معرض کرونا هستند 
score: 0.51240385 

9 : آیا اختلالات تیروئید ریسک ابتلا به کروناویروس را افزایش می‌دهد 
score: 0.47757724 

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
question 1: من باردار هستم چگونه می‌توانم از خودم در مقابل بی

{'MAP@10': 0.7513611205593348,
 'MP@10': 0.7,
 'Mean NDCJ': 0.9447589165551628,
 'mean of the mean score of all questions scale 0-1': 0.50717723,
 'mean of the mean score of all questions scale 0-4': 2.5064713094975626,
 'predictions scale 1': [[0.50650954,
   0.51525116,
   0.5784531,
   0.47669148,
   0.52330977,
   0.4945562,
   0.66398525,
   0.4719077,
   0.51240385,
   0.47757724],
  [0.5828557,
   0.49264938,
   0.4554768,
   0.6942357,
   0.6690251,
   0.5931834,
   0.4897256,
   0.6119787,
   0.40197968,
   0.6790132],
  [0.6427395,
   0.21443713,
   0.50051177,
   0.44440496,
   0.3450975,
   0.60653985,
   0.6994442,
   0.48385352,
   0.49249318,
   0.335728],
  [0.827912,
   0.8320115,
   0.7127556,
   0.6783359,
   0.60410446,
   0.69808435,
   0.586103,
   0.66480803,
   0.47949198,
   0.45573378],
  [0.28504768,
   0.258583,
   0.3353686,
   0.2652994,
   0.38376275,
   0.3592863,
   0.15485913,
   0.28941998,
   0.2358166,
   0.37540403],
  [0.689268,
   0.75769186,
   

In [ ]:
TFIDF_results = show_results_with_records('sbert-WikiNli-FineTuned', top_n=10, results_path=results_path, minimum_score_for_precision=0.4) # new questions
TFIDF_results

Model:  sbert-WikiNli-FineTuned
question 0: آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0 : سیگاری‌ها بیشتر در خطر ابتلا به کرونا هستند 
score: 0.50650954 

1 : مصرف‌کنندگان موادمخدر بیشتر در معرض کرونا هستند 
score: 0.51240385 

2 : احتمال ابتلای‌ قدبلندها به کرونا بیشتر است 
score: 0.4719077 

3 : احتمال ابتلا به کرونا در افراد طاس بیشتر است 
score: 0.52330977 

4 : آیا سیگاری‌ها معتادان به کرونا مبتلا نمی‌شوند 
score: 0.38531637 

5 : مرگ میر بر اثر کرونا در کدام بیماران بیشتر است 
score: 0.30853167 

6 : ۴۲ درصد مبتلایان کرونا جوان هستند 
score: 0.40496105 

7 : وجود بیماری‌های زمینه‌ای در ۷۵ درصد از جوانان فوت شده بر اثر کرونا 
score: 0.23662229 

8 : کدام گروه سنی بیشتر در معرض خطر کروناست 
score: 0.66398525 

9 : آیا اختلالات تیروئید ریسک ابتلا به کروناویروس را افزایش می‌دهد 
score: 0.47757724 

question 1: من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
0 : طلوع زندگی پس از کرونا 
score: 0.34750092 

1 : زنان باردار درمقابل کرونا چه کارها

{'average number of records with score more than 0.4': 6.8,
 'mean of precisions in identifying records more than0.4': 0.68,
 'mean of the mean score of all questions': 0.49463657,
 'predictions': [[0.50650954,
   0.51240385,
   0.4719077,
   0.52330977,
   0.38531637,
   0.30853167,
   0.40496105,
   0.23662229,
   0.66398525,
   0.47757724],
  [0.34750092,
   0.5828557,
   0.28390542,
   0.42512608,
   0.49080524,
   0.5609355,
   0.51428175,
   0.18912816,
   0.25854045,
   0.53925484],
  [0.44440496,
   0.6994442,
   0.60653985,
   0.6427395,
   0.5523268,
   0.3450975,
   0.4089861,
   0.50051177,
   0.21443713,
   0.274422],
  [0.7127556,
   0.6783359,
   0.827912,
   0.45573378,
   0.586103,
   0.66480803,
   0.8320115,
   0.453519,
   0.5316352,
   0.6910368],
  [0.7688628,
   0.37471625,
   0.26482502,
   0.3381585,
   0.7981358,
   0.46681684,
   0.3403681,
   0.76257306,
   0.49111,
   0.5253413],
  [0.76480377,
   0.7135135,
   0.6978076,
   0.469115,
   0.7891636,
   0.757

# Novel approach experiment

In [ ]:


novel_4 = show_results_with_records('Ranked_BM25_TFIDF_sbert-WikiNli-DifferentWeights_FineTuned_f300_Ranked_2-0.9s', top_n=10, results_path=results_path, minimum_score_for_precision=0.4, print_output=True) 
novel_4

Model:  Ranked_BM25_TFIDF_sbert-WikiNli-DifferentWeights_FineTuned_f300_Ranked_2-0.9s
[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
question 0: آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0 : سیگاری‌ها بیشتر در خطر ابتلا به کرونا هستند 
score: 0.44328672 

1 : خطر مرگ ابتلا به کرونا در کودکان چاق بیشتر است 
score: 0.41301087 

2 : خطر ابتلا به سکته مغزی در افراد سیگاری مبتلا به کرونا بیشتر است 
score: 0.4041261 

3 : احتمال ابتلا به کرونا در افراد طاس بیشتر است 
score: 0.4470177 

4 : آیا کودکان چاق مستعد ابتلا به کرونا هستند 
score: 0.5167437 

5 : خطر ابتلا به کرونا در دیابتی‌ها برابر دیگران است 
score: 0.55244297 

6 : معرفی پرریسک‌ترین افراد در معرض خطر ابتلا به کرونا 
score: 0.41943002 

7 : کدام گروه سنی بیشتر در معرض خطر کروناست 
score: 0.6187066 

8 : احتمال ابتلای‌ قدبلندها به کرونا بیشتر است 
score: 0.33502787 

9 : علت پایین بودن آمار کودکان مبتلا به کرونا سهم بالای مردان در ابتلا مرگ بیماری 
score: 0.39962837 

[1, 1, 1, 0, 1, 1, 0, 1, 1, 1]
question 1: من باردار هستم چگ

{'MAP@10': 0.8233035241874528,
 'MP@10': 0.7340000000000001,
 'Mean NDCJ': 0.9492349255460073,
 'mean of the mean score of all questions scale 0-1': 0.5141083,
 'mean of the mean score of all questions scale 0-4': 2.559494952022574}

# Human Judgement


In [ ]:
#title Default title text
BM25_results = show_results_with_records('BM25', top_n=10, results_path=results_path, minimum_score_for_precision=0.4, print_output=True) 
BM25_results

Model:  BM25
[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
question 0: آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0 : تزریق واکسن کرونا در بارداری بی خطر است 
score: 0.5014234 

1 : آیا کرونا از مادر به جنین منتقل می‌شود 
score: 0.6426012 

2 : حفظ سلامتی زنان باردار در دوران کرونا 
score: 0.6387498 

3 : کرونا از مادر باردار به جنین منتقل می‌شود 
score: 0.6703031 

4 : کاهش فعالیت ورزشی زنان باردار افزایش افسردگی در همه‌گیری کرونا 
score: 0.44684935 

5 : تاثیر ویروس کرونا بر زنان باردار جنین 
score: 0.64813954 

6 : توصیه‌های کرونایی به زنان باردار 
score: 0.63169926 

7 : مصرف روی آهن در دوران نقاهت کرونا الزامی است 
score: 0.10569226 

8 : خانم‌های باردار کرونا 
score: 0.65867096 

9 : توصیه هایی برای پیشگیری از کرونا در زنان باردار 
score: 0.61621755 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
question 1: من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
0 : مشاهدات میدانی از آی سی یو بخش ویژه کرونا در بیمارستان امام حسینع 
score: 0.059244193 

1 : مصائب یک لقمه نان ک

{'MAP@10': 0.5996788958175863,
 'MP@10': 0.4040000000000001,
 'Mean NDCJ': 0.9010384264535858,
 'mean of the mean score of all questions scale 0-1': 0.3758390493917866,
 'mean of the mean score of all questions scale 0-4': 2.0156932520338366}

In [ ]:
TFIDF_results = show_results_with_records('TFIDF', top_n=10, results_path=results_path, minimum_score_for_precision=0.4, print_output=True) 
TFIDF_results

Model:  TFIDF
[1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
question 0: آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0 : تزریق واکسن کرونا در بارداری بی خطر است 
score: 0.5014234 

1 : خطرات تجویز داروهای کرونا برای زنان باردار 
score: 0.55455256 

2 : زنان باردار درمقابل کرونا چه کارهایی باید انجام دهند 
score: 0.69304454 

3 : کروناویروس جدید از مادر به جنین منتقل نمی‌شود 
score: 0.40355647 

4 : مطالعات جدید اثر کرونا بر زنان باردار به چه صورت است 
score: 0.6646314 

5 : توصیه‌های وزارت بهداشت به زنان باردار در روزهای کرونایی 
score: 0.63442457 

6 : توصیه هایی برای پیشگیری از کرونا در زنان باردار 
score: 0.61621755 

7 : کرونا لزوم مراقبت‌های بهداشتی در دوران بارداری 
score: 0.4967451 

8 : آیا بهبودیافتگان کرونا هم باید واکسن بزنند 
score: 0.20298852 

9 : آیا کرونا از مادر به جنین منتقل می‌شود 
score: 0.6426012 

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
question 1: من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
0 : ۷ باور غلط که در روز‌های کرونایی سلامتی‌تان را تهد

{'MAP@10': 0.6797175012597633,
 'MP@10': 0.49200000000000005,
 'Mean NDCJ': 0.9152863941188214,
 'mean of the mean score of all questions scale 0-1': 0.40389952309069627,
 'mean of the mean score of all questions scale 0-4': 2.112362809383035}

In [ ]:
sbert_wikinli_results = show_results_with_records('sbert-WikiNli', top_n=10, results_path=results_path, minimum_score_for_precision=0.4,print_output=True) 
sbert_wikinli_results

Model:  sbert-WikiNli
[0, 1, 1, 1, 0, 0, 0, 0, 1, 0]
question 0: آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0 : سیگاری‌ها بیشتر در خطر ابتلا به کرونا هستند 
score: 0.39186615 

1 : مصرف‌کنندگان موادمخدر بیشتر در معرض کرونا هستند 
score: 0.42245016 

2 : احتمال ابتلا به کرونا در افراد طاس بیشتر است 
score: 0.42972735 

3 : کدام گروه سنی بیشتر در معرض خطر کروناست 
score: 0.59043175 

4 : احتمال ابتلای‌ قدبلندها به کرونا بیشتر است 
score: 0.24366747 

5 : خطر ابتلا به کرونا در افراد کم خواب 
score: 0.33296338 

6 : کودکان کرونا نمی‌گیرند این باور غلط آنها را به خطر می‌اندازد 
score: 0.21350585 

7 : ۴۲ درصد مبتلایان کرونا جوان هستند 
score: 0.237025 

8 : امکان ابتلای شدید به کرونا برای کسانی که قبلا مبتلا شده‌اند 
score: 0.42174792 

9 : وجود بیماری‌های زمینه‌ای در ۷۵ درصد از جوانان فوت شده بر اثر کرونا 
score: 0.09908013 

[0, 0, 0, 1, 0, 0, 1, 1, 0, 0]
question 1: من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
0 : طلوع زندگی پس از کرونا 
score:

{'MAP@10': 0.6319355914588057,
 'MP@10': 0.45799999999999996,
 'Mean NDCJ': 0.904549673874681,
 'mean of the mean score of all questions scale 0-1': 0.3947285263406758,
 'mean of the mean score of all questions scale 0-4': 2.090661209501484}

In [ ]:
sbert_WikiNli_Finetuned_results = show_results_with_records('sbert-WikiNli-FineTuned', top_n=10, results_path=results_path,print_output=True, minimum_score_for_precision=0.4) 
sbert_WikiNli_Finetuned_results

Model:  sbert-WikiNli-FineTuned
[0, 1, 0, 1, 0, 0, 0, 0, 1, 0]
question 0: آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0 : سیگاری‌ها بیشتر در خطر ابتلا به کرونا هستند 
score: 0.39186615 

1 : مصرف‌کنندگان موادمخدر بیشتر در معرض کرونا هستند 
score: 0.42245016 

2 : احتمال ابتلای‌ قدبلندها به کرونا بیشتر است 
score: 0.24366747 

3 : احتمال ابتلا به کرونا در افراد طاس بیشتر است 
score: 0.42972735 

4 : آیا سیگاری‌ها معتادان به کرونا مبتلا نمی‌شوند 
score: 0.31413445 

5 : مرگ میر بر اثر کرونا در کدام بیماران بیشتر است 
score: 0.26708597 

6 : ۴۲ درصد مبتلایان کرونا جوان هستند 
score: 0.237025 

7 : وجود بیماری‌های زمینه‌ای در ۷۵ درصد از جوانان فوت شده بر اثر کرونا 
score: 0.09908013 

8 : کدام گروه سنی بیشتر در معرض خطر کروناست 
score: 0.59043175 

9 : آیا اختلالات تیروئید ریسک ابتلا به کروناویروس را افزایش می‌دهد 
score: 0.35286617 

[0, 1, 0, 0, 1, 1, 0, 0, 0, 1]
question 1: من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
0 : طلوع زندگی پس از کرون

{'MAP@10': 0.6060740362811792,
 'MP@10': 0.44799999999999995,
 'Mean NDCJ': 0.9093277158787931,
 'mean of the mean score of all questions scale 0-1': 0.3828307600354129,
 'mean of the mean score of all questions scale 0-4': 2.056353738001515}

In [ ]:
ranked_results = show_results_with_records('Ranked_BM25_TFIDF_SBERT_Wikinli_Fine-tuned_Different-Weights', top_n=10, results_path=results_path, minimum_score_for_precision=0.4,print_output=True) 
ranked_results

Model:  Ranked_BM25_TFIDF_SBERT_Wikinli_Fine-tuned_Different-Weights
[0, 1, 1, 0, 1, 0, 1, 0, 1, 0]
question 0: آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
0 : سیگاری‌ها بیشتر در خطر ابتلا به کرونا هستند 
score: 0.39186615 

1 : خطر مرگ ابتلا به کرونا در کودکان چاق بیشتر است 
score: 0.410074 

2 : آیا کودکان چاق مستعد ابتلا به کرونا هستند 
score: 0.5221499 

3 : خطر ابتلا به سکته مغزی در افراد سیگاری مبتلا به کرونا بیشتر است 
score: 0.3758938 

4 : احتمال ابتلا به کرونا در افراد طاس بیشتر است 
score: 0.42972735 

5 : معرفی پرریسک‌ترین افراد در معرض خطر ابتلا به کرونا 
score: 0.39135662 

6 : کدام گروه سنی بیشتر در معرض خطر کروناست 
score: 0.59043175 

7 : احتمال ابتلای‌ قدبلندها به کرونا بیشتر است 
score: 0.24366747 

8 : مصرف‌کنندگان موادمخدر بیشتر در معرض کرونا هستند 
score: 0.42245016 

9 : آیا اختلالات تیروئید ریسک ابتلا به کروناویروس را افزایش می‌دهد 
score: 0.35286617 

[1, 1, 0, 1, 1, 1, 0, 1, 0, 1]
question 1: من باردار هستم چگونه می‌توانم از خودم در مقابل بیم

{'MAP@10': 0.5837573948097757,
 'MP@10': 0.466,
 'Mean NDCJ': 0.9258421734845793,
 'mean of the mean score of all questions scale 0-1': 0.4029630167173231,
 'mean of the mean score of all questions scale 0-4': 2.1143121514163505}